In [1]:
%load_ext cython

In [2]:
%%cython
cpdef mandelbrot( int[:,::1] Z, float left, float right, float bottom, float top, 
                 float d, int numx, int numy, int maxIter):
    cdef complex z, c
    cdef int x, y, it
    for x in range(numx):
        for y in range(numy):
            c = left + x * d + 1j*( bottom + y * d)
            z = c
            for it in range(maxIter):
                if z.real**2+z.imag**2 >= 4:
                    break
                else:
                    z = z*z + c
            Z[x,y] = it

In [3]:
%autosave 0
%matplotlib qt5
#widget tk qt5
import matplotlib.pyplot as plt
import numpy as np

Autosave disabled


In [4]:

def init_data():
    left, right = -2, 2
    bottom, top = -2, 2
    dx = np.abs(right-left)
    dy = np.abs(top-bottom)    
    d = 0.1 # must be even
    numx=int(dx/d)
    numy=int(dy/d)
    #d,dx,num
    x = np.linspace(left, right, numx+1)
    y = np.linspace(bottom, top, numy+1)
    z = np.zeros((numx,numy), dtype=np.int32)
    
    maxIter=63
    mandelbrot(z, left, right, bottom, top, d, numx, numy, maxIter)
    #z = f(x,y)
    return x, y, z, numx, numy, d, maxIter

def doZoom( ex, ey, x, y):
    nix = np.argmin(np.abs(ex-x))
    niy = np.argmin(np.abs(ey-y))
    left, right = x[0], x[-1]
    bottom, top = y[0], y[-1]
    arm_x=(right-left)/4
    arm_y=(top-bottom)/4
    nr=np.argmin(np.abs(x-(x[nix]+arm_x)))
    nl=np.argmin(np.abs(x-(x[nix]-arm_x)))
    nt=np.argmin(np.abs(y-(y[niy]+arm_y)))
    nb=np.argmin(np.abs(y-(y[niy]-arm_y)))
    return np.linspace(x[nl],x[nr],numx+1), np.linspace(y[nb],y[nt],numy+1)

In [12]:
state={ 'pressId':False, 'releaseId':False, 'motionId':False, 'lastx':False, 'lasty':False}

def motion_handler(event):
    plt.ioff()
    global x,y,z
    if state['lastx']!=False:
        mouse_travel = np.linalg.norm( [ state['lastx'] - event.x, state['lasty'] - event.y ] )
        if mouse_travel > 3:
            if event.button==1:
                print('btn1, x({:.3},{:.3}), y({:.3},{:.3}), z({},{})'.format(x.min(), x.max(), y.min(), y.max(), z.min(), z.max()))
                #tb.set_text('Button:1, m.travel:{:.3}'.format(mouse_travel))
                x, y = doZoom( event.xdata, event.ydata, x, y)
                print('event data({},{})'.format(event.xdata, event.ydata))
                z = np.zeros((numx,numy), dtype=np.int32)
                mandelbrot(z, x[0], x[-1], y[0], y[-1], d, numx, numy, maxIter)
                #mandelbrot(z,x,y,63)
                #z = f(x,y)
                ax.cla()
                #ax.set_xlim(x[0],x[-1])
                #ax.set_ylim(y[0],y[-1])
                #qm=ax.pcolormesh(y,x,z)
                qm=ax.pcolormesh(x,y,z.transpose())
                #im = ax.imshow(z.transpose(), origin='upper', extent=[x[0],x[-1],y[0],y[-1]])
                #tb.set_text('z min:{:.3}, max:{:.3}'.format(z.min(), z.max()))
                tb.set_text('x({:.3},{:.3}), y({:.3},{:.3}), z({},{})'.format(x.min(), x.max(), y.min(), y.max(), z.min(), z.max()))
                #print('x({:.3},{:.3}),y({:.3},{:.3}),z({:.3},{:.3})'.format(x.min(), x.max(), y.min(), y.max(), z.min(), z.max()))
                #print('y min:{:.3}, max:{:.3}'.format(y.min(), y.max()))
                #print('x min:{:.3}, max:{:.3}'.format(x.min(), x.max()))
            if event.button==3:
                print('button 3')
                #tb.set_text('Button:3, m.travel:{:.3}'.format(mouse_travel))
    state['lastx']=event.x
    state['lasty']=event.y
    event.canvas.draw()
    plt.ion()
        
def button_press_handler(event):
    print('PRESS', end=';')
    state['motionId']  = fig.canvas.mpl_connect('motion_notify_event'  , motion_handler)
    state['releaseId'] = fig.canvas.mpl_connect('button_release_event' , button_release_handler)
    
def button_release_handler(event):
    print('RELEASE', end=';')
    fig.canvas.mpl_disconnect(state['motionId'])
    
def axes_enter_handler(event):
    print('ENTER', end=';')
    #tb.set_text('Press left or right mouse button')
    state['pressId'] = fig.canvas.mpl_connect('button_press_event'  , button_press_handler)
    event.canvas.draw()

def axes_leave_handler(event):
    print('LEAVE', end=';')
    #tb.set_text('Left...')
    fig.canvas.mpl_disconnect(state['pressId'])
    fig.canvas.mpl_disconnect(state['releaseId'])
    fig.canvas.mpl_disconnect(state['motionId'])
    event.canvas.draw()

In [13]:
x, y, z, numx, numy, d, maxIter = init_data()

fig, ax = plt.subplots()
fig.canvas.mpl_connect('axes_enter_event',axes_enter_handler)
fig.canvas.mpl_connect('axes_leave_event',axes_leave_handler)
tb = fig.text(x=0.6,y=0.9,s='place holder')
#ax.set_xlim(x[0],x[-1])
#ax.set_ylim(y[0],y[-1])
#qm=ax.pcolormesh(y,x,z)
qm = ax.pcolormesh(x,y,z.transpose())
#im = ax.imshow(z.transpose(), origin='upper', extent=[x[0],x[-1],y[0],y[-1]])
plt.show()

ENTER;ENTER;

In [9]:
z[::10],x,y

(array([[62, 62, 62, 62, 62, 62, 62, 62,  7,  6,  4,  3,  2,  1,  1,  1,
          1,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0]], dtype=int32),
 array([0.05     , 0.0515625, 0.053125 , 0.0546875, 0.05625  , 0.0578125,
        0.059375 , 0.0609375, 0.0625   , 0.0640625, 0.065625 , 0.0671875,
        0.06875  , 0.0703125, 0.071875 , 0.0734375, 0.075    , 0.0765625,
        0.0